## Tide gauge data processing
#### 1. Convert the .txt file (tide gauge station data from Marine Department) to .nc file
#### 2. Update the station name of the gauge station data from Obversatory.
#### 3. Merge the tgauge stations data from Marine Department and Obversatory, respectively

### Tgauge stations data from Marine Department

In [1]:
import sys
sys.path.append("../utils")
from get_filesPath import get_filesPath
import numpy as np
import xarray as xr


### Postition of the tide stations
lat_cc, lon_cc = np.array([22.214]), np.array([114.022])  
lat_klw, lon_klw = np.array([22.459]), np.array([114.361])  
lat_kc, lon_kc = np.array([22.324]), np.array([114.122])  
lat_mw, lon_mw = np.array([22.364]), np.array([114.071])  

### Super-parameter

In [2]:
root_dir = os.path.dirname(os.getcwd())
name = ['cc']
long_name = 'Cheung Chau from Hong Kong Marine Department'
lat, lon = np.array([22.214]), np.array([114.022])
dir_md_tgauge = root_dir+'/data/tgauge_data/marine_department_hk/cheung_chau(cc)'
pat_to_save = dir_md_tgauge+'/tgauge_cc.nc'
print(dir_md_tgauge)


/Users/luo/OneDrive/SAR-Altimetry/sentinel3-altimetry-l2/data/tgauge_data/marine_department_hk/cheung_chau(cc)


### Extract the time and sea level from the tige files

In [3]:
## get files path name
files_path = get_filesPath(dir_md_tgauge, key_words='.txt')
files_path = sorted(files_path)
print('.txt files number:', len(files_path))
times = []
sea_level = []
for file_path in files_path:
    f = open(file_path,"r")
    text = f.read()
    text = text.split('\n')
    for i in range(len(text)-1):
        time_i = np.datetime64(text[i][0:4]+'-'+text[i][5:7]+'-'+text[i][8:19])
        sea_level_i = float(text[i][20:])
        times.append(time_i)
        sea_level.append(sea_level_i)

print(len(times))
print(len(sea_level))


.txt files number: 1827
263088
263088


### Save as the xarray format

In [4]:
dset = xr.Dataset({
    'sea_level': xr.DataArray(
        data = sea_level,
        dims = ['time'],
        coords = {'time': times},
    ),
    'lat': xr.DataArray(
        data = lat,
        dims= 'record_id',
        coords = {'record_id': name},
    ),
    'lon': xr.DataArray(
        data = lon,
        dims= 'record_id',
        coords = {'record_id': name},
    ),
    'station_name': xr.DataArray(
        data = name,
        dims= 'record_id',
        coords = {'record_id': name},
        attrs={'long_name': long_name}
    )})
dset


<xarray.Dataset>
Dimensions:       (record_id: 1, time: 263088)
Coordinates:
  * time          (time) datetime64[ns] 2016-01-01 ... 2020-12-31T23:50:00
  * record_id     (record_id) <U2 'cc'
Data variables:
    sea_level     (time) float64 1.95 1.98 1.99 1.99 ... 2.56 2.53 2.48 2.43
    lat           (record_id) float64 22.21
    lon           (record_id) float64 114.0
    station_name  (record_id) <U2 'cc'

#### Save the tgauge data

In [5]:
# dset.to_netcdf(path=pat_to_save)


### Merge all the tgauge stations data from Marine Department

In [78]:
path_md_cc = root_dir+'/data/tgauge_data/marine_department_hk/cheung_chau(cc)/tgauge_cc.nc'
path_md_klw = root_dir+'/data/tgauge_data/marine_department_hk/ko_lau_wan(klw)/tgauge_klw.nc'
path_md_kc = root_dir+'/data/tgauge_data/marine_department_hk/kwai_chung(kc)/tgauge_kc.nc'
path_md_mw = root_dir+'/data/tgauge_data/marine_department_hk/ma_wan(mw)/tgauge_mw.nc'
path_md_save = root_dir+'/data/tgauge_data/marine_department_hk/tgauge_md.nc'

In [87]:
tgauge_cc = xr.open_dataset(path_md_cc)
tgauge_klw = xr.open_dataset(path_md_klw)
tgauge_kc = xr.open_dataset(path_md_kc)
tgauge_mw = xr.open_dataset(path_md_mw)
tgauge_qb = xr.open_dataset(path_obser_qb)
tgauge_all = xr.concat([tgauge_cc, tgauge_klw, tgauge_kc, tgauge_mw], dim="record_id")
tgauge_all['station_name'].attrs['long_name'] = 'Tide gauge data from Hong Kong Marine Department'
tgauge_all


<xarray.Dataset>
Dimensions:       (record_id: 4, time: 263088)
Coordinates:
  * time          (time) datetime64[ns] 2016-01-01 ... 2020-12-31T23:50:00
  * record_id     (record_id) object 'cc' 'klw' 'kc' 'mw'
Data variables:
    sea_level     (record_id, time) float64 1.95 1.98 1.99 ... 2.59 2.55 2.5
    lat           (record_id) float64 22.21 22.5 22.32 22.36
    lon           (record_id) float64 114.0 114.4 114.1 114.1
    station_name  (record_id) object 'cc' 'klw' 'kc' 'mw'

In [86]:
tgauge_all.to_netcdf(path=path_md_save)


### For the tgauge station data from Hong Kong Observatory
#### 1. Update the tgauge data from Observatory.
#### 2. Merge all the tgauge data from Observatory (future).

In [89]:
path_obser_qb = root_dir+'/data/tgauge_data/observatory_hk/tgauge_qb.nc'
path_qb_save = root_dir+'/data/tgauge_data/observatory_hk/tgauge_qb_new.nc'
tgauge_qb['station_name'].values = ['qb']
tgauge_qb['station_name'].attrs['long_name'] = 'Quarry Bay from Hong Kong Observatory'
tgauge_qb.to_netcdf(path=path_qb_save)
